In [1]:
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.layers import Dot
# ====== 4. 标准化 ======
from sklearn.preprocessing import StandardScaler

import shap

from statsmodels.tsa.stattools import adfuller
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns 
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Attention, Bidirectional, Flatten

from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Attention, Bidirectional, Flatten

2025-05-03 13:55:25.238335: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-03 13:55:25.250152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746251725.262632    1395 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746251725.266407    1395 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746251725.276717    1395 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
import os
# 屏蔽所有 GPU，强制 CPU 模式
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf

In [12]:
import h5py

# 打开H5文件
with h5py.File("./mode/cnn_lstm_model.h5", "r") as f:
    # 遍历模型层信息
    model_config = f.attrs["model_config"]
    print("模型结构配置:", model_config)

模型结构配置: {"class_name": "Functional", "config": {"name": "functional", "trainable": true, "layers": [{"class_name": "InputLayer", "config": {"batch_shape": [null, 60, 24], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_layer_5"}, "name": "input_layer_5", "inbound_nodes": []}, {"class_name": "Conv1D", "config": {"name": "conv1d_5", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "filters": 64, "kernel_size": [3], "strides": [1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1], "groups": 1, "activation": "relu", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "class_name": "Zeros", "config": {}, "registered_name": null}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": nu

In [15]:
import tensorflow as tf

# 尝试加载 .keras 文件
try:
    model = tf.keras.models.load_model("./mode/cnn_lstm_model.keras",safe_mode=False  )
    print("模型加载成功！")
except Exception as e:
    print(f"加载失败，错误信息: {e}")

加载失败，错误信息: Exception encountered when calling Lambda.call().

We could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None, 29, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_77>',)
  • kwargs={'mask': 'None'}


In [20]:
with h5py.File("./mode/fixed_cnn_lstm_model3.h5", "r") as f:
    # 遍历模型层信息
    model_config = f.attrs["model_config"]
    print("模型结构配置:", model_config)

模型结构配置: {"class_name": "Functional", "config": {"name": "functional_3", "trainable": true, "layers": [{"class_name": "InputLayer", "config": {"batch_shape": [null, 60, 9], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_layer_4"}, "name": "input_layer_4", "inbound_nodes": []}, {"class_name": "Conv1D", "config": {"name": "conv1d_3", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "filters": 64, "kernel_size": [3], "strides": [1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1], "groups": 1, "activation": "relu", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "class_name": "Zeros", "config": {}, "registered_name": null}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": n

In [21]:
import tensorflow as tf

# 临时禁用安全模式以加载旧模型
tf.keras.config.enable_unsafe_deserialization()
#ld_model = tf.keras.models.load_model("./mode/cnn_lstm_model.keras")
ld_model = tf.keras.models.load_model("./mode/fixed_cnn_lstm_model3.keras")


# 提取模型配置
model_config = ld_model.get_config()

In [22]:
xgb_model = xgb.XGBRegressor()
xgb_model.load_model("./mode/xgb_model.json")

In [28]:
import pandas as pd
from scipy.stats import zscore

real_time_df = pd.read_csv("./data/merge.csv", parse_dates=["datetime"])
real_time_df = real_time_df.sort_values("datetime")

# # 处理缺失值
# real_time_df = real_time_df.ffill()

# # 剔除异常值（以收盘价为例）
# z_scores = zscore(real_time_df["close"])
# real_time_df = real_time_df[(z_scores.abs() < 3)]

In [29]:
real_time_df.head()

,Unnamed: 0,datetime,contract,symbol,exchange,open,high,low,close,openinterest,...,Signal,Volatility_10,Volume_Change,Price_Volume_Corr,future_close,涨跌幅,hour,minute,session,涨跌幅_scaled
0,13,2017-01-03 09:13:00,HC1705,HC,SHFE,3372.0,3372.0,3362.0,3363.0,172022.0,...,15.745432,0.757838,-0.013410,0.196844,3356.0,-0.208147,9,13,Morning,-0.408166
1515044,3279334,2017-01-03 09:13:00,RB1705,RB,SHFE,2944.0,2945.0,2933.0,2933.0,1160893.0,...,15.191008,0.626449,-0.036573,0.106531,2925.0,-0.272758,9,13,Morning,-0.532414
505527,1093120,2017-01-03 09:13:00,I1705,I,DCE,560.5,560.5,558.0,558.0,439663.0,...,2.528772,-0.294013,-0.019669,0.142922,556.5,-0.268817,9,13,Morning,-0.524835
2474728,5465548,2017-01-03 09:13:00,SM1705,SM,CZCE,6936.0,6936.0,6920.0,6928.0,9811.0,...,16.054833,0.334108,0.738855,-0.092269,6804.0,-1.771117,9,13,Morning,-3.413782
1013680,2186227,2017-01-03 09:13:00,JM1705,JM,DCE,1184.5,1185.0,1180.0,1180.5,75153.0,...,3.461766,-0.093331,0.068832,0.076725,1169.0,-0.974163,9,13,Morning,-1.881227


In [30]:
real_time_df["MA5"]

0         -0.417443
1515044   -0.520015
505527    -1.089222
2474728    0.432299
1013680   -0.940232
             ...   
2020637   -0.486508
1515043   -0.996291
505526    -0.463264
2474727    0.133794
3318337    1.830556
Name: MA5, Length: 3318338, dtype: float64

In [36]:
# 假设需要预测的时间点为2025-04-18 14:59:00
prediction_time = pd.Timestamp("2025-04-17 9:00:00")
input_window_start = prediction_time - pd.Timedelta(minutes=60)

# 提取数据窗口
input_data = real_time_df[
    (real_time_df["datetime"] >= input_window_start) &
    (real_time_df["datetime"] < prediction_time)
].sort_values("datetime")



In [37]:
input_data

,Unnamed: 0,datetime,contract,symbol,exchange,open,high,low,close,openinterest,...,Signal,Volatility_10,Volume_Change,Price_Volume_Corr,future_close,涨跌幅,hour,minute,session,涨跌幅_scaled
